In [1]:
'''
Name   : Rupesh Garsondiya
github : @Rupeshgarsondiya
Topic  : ANN using torch in Deep-Learning(DL)
'''

'\nName   : Rupesh Garsondiya\ngithub : @Rupeshgarsondiya\nTopic  : ANN using torch in Deep-Learning(DL)\n'

In [32]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [33]:
# set random seeds for reproductibility
torch.manual_seed(42)

In [6]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d zalando-research/fashionmnist

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
 60% 41.0M/68.8M [00:00<00:00, 209MB/s]
100% 68.8M/68.8M [00:00<00:00, 235MB/s]


In [7]:
import zipfile
zipfile = zipfile.ZipFile('/content/fashionmnist.zip')
zipfile.extractall('/content')
zipfile.close()

In [48]:
df_train = pd.read_csv('/content/fashion-mnist_train.csv')
df_train.head()

df_test = pd.read_csv('/content/fashion-mnist_test.csv')
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# Extract features and labels for training set
# X_train: All columns except 'label' from the df_train DataFrame are used as features for the training set
# y_train: The 'label' column from df_train DataFrame is used as the target labels for the training set
X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values

# Extract features and labels for test set
# X_test: All columns except 'label' from the df_test DataFrame are used as features for the test set
# y_test: The 'label' column from df_test DataFrame is used as the target labels for the test set
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Print the shape (dimensions) of the training and testing data
# This helps confirm the structure and size of the datasets
print('X_train : ', X_train.shape)  # Prints the shape of the training features
print('y_train : ', y_train.shape)  # Prints the shape of the training labels
print('X_test : ', X_test.shape)    # Prints the shape of the test features
print('y_test : ', y_test.shape)    # Prints the shape of the test labels


X_train :  (60000, 784)
y_train :  (60000,)
X_test :  (10000, 784)
y_test :  (10000,)


In [50]:
# Normalize the pixel value
X_train = X_train / 255.0
X_test = X_test / 255.0

In [51]:
# Define a custom dataset class inheriting from PyTorch's Dataset class
class CustomDataset(Dataset):

  # Initialization method (__init__) to store the features and targets
  # features: Input features of the dataset (e.g., image data, tabular data)
  # targets: Corresponding labels or target values for the features
  def __init__(self, features, targets):
    # Convert features and targets to torch tensors with appropriate data types
    self.features = torch.tensor(features, dtype=torch.float32)  # Features are stored as float32 tensors
    self.targets = torch.tensor(targets, dtype=torch.long)  # Targets are stored as long (integer) tensors

  # Method to return the number of samples in the dataset
  def __len__(self):
    return self.features.shape[0]  # The length is equal to the number of rows in the features array

  # Method to retrieve a sample by index
  # idx: Index of the data sample to retrieve
  def __getitem__(self, idx):
    return self.features[idx], self.targets[idx]  # Return the features and targets for the given index


In [52]:
# create train dataset object
train_dataset = CustomDataset(X_train,y_train)

In [53]:
# create test dataset object
test_dataset = CustomDataset(X_test,y_test)

In [54]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [86]:
# Define a custom neural network class inheriting from PyTorch's nn.Module
class NN(nn.Module):

  # Initialization method (__init__) to define the layers and architecture of the neural network
  # input_features: Number of input features (e.g., the number of features in the dataset)
  def __init__(self, input_features):
    # Call the parent class constructor
    super().__init__()

    # Define the layers of the neural network using nn.Sequential
    self.model = nn.Sequential(
        nn.Linear(input_features, 128),  # First fully connected layer (input -> 128 units)
        nn.ReLU(),                      # ReLU activation function (introduces non-linearity)
        nn.Linear(128, 64),              # Second fully connected layer (128 -> 64 units)
        nn.ReLU(),                      # ReLU activation function
        nn.Linear(64, 10)                # Final fully connected layer (64 -> 10 units for classification)
        # Note: The final layer doesn't need an explicit softmax since PyTorch's sparse categorical cross-entropy loss
        # handles the softmax activation internally during the loss calculation.
    )

  # Define the forward pass of the network
  # x: The input data (e.g., features of a sample)
  def forward(self, x):
    # Pass the input through the network and return the output
    return self.model(x)


In [80]:

# set learnign rate and epochs
epochs = 10
learning_rate = 0.01


In [81]:
# Instantiate the neural network model with the number of input features
# X_train.shape[1] gives the number of features in the input data (i.e., number of input units in the first layer)
model = NN(X_train.shape[1])

# Define the loss function as CrossEntropyLoss
# CrossEntropyLoss is commonly used for multi-class classification tasks.
# It combines softmax activation and the negative log-likelihood loss in one function.
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer using Adam
# Adam is an optimization algorithm that adjusts learning rates based on the first and second moments of the gradients.
# We are passing model.parameters() to let the optimizer know which parameters to update during training.
# The learning rate (lr) controls the step size for each parameter update.
optimizer = Adam(model.parameters(), lr=learning_rate)


In [82]:
# Training Loop

# Iterate through the specified number of epochs
for epochs in range(epochs):

    # Initialize variable to track total loss for this epoch
    total_epochs_loss = 0

    # Iterate through each batch of data in the train_loader (this handles batching and shuffling of the data)
    for batch_feature, label in train_loader:

        # Forward pass: Pass the batch of features through the model to get the predictions (output)
        output = model(batch_feature)

        # Calculate the loss between the predicted output and true labels
        loss = loss_fn(output, label)

        # Add the current loss to the total loss for the epoch
        total_epochs_loss += loss.item()

        # Backward pass: Compute gradients of the loss with respect to model parameters
        optimizer.zero_grad()  # Clear the previous gradients
        loss.backward()        # Backpropagate the loss to compute new gradients

        # Update model parameters (weights and biases) using the optimizer
        optimizer.step()

    # Calculate the average loss for the epoch (divide by the number of batches in the train_loader)
    avg_loss = total_epochs_loss / len(train_loader)

    # Print the average loss for the current epoch
    print(f'Epoch : {epochs+1} Loss : {avg_loss}')


Epoch : 1 Loss : 0.5391553043146928
Epoch : 2 Loss : 0.4403197630286217
Epoch : 3 Loss : 0.4120727299749851
Epoch : 4 Loss : 0.40670799601276714
Epoch : 5 Loss : 0.3932236555973689
Epoch : 6 Loss : 0.39020857316652935
Epoch : 7 Loss : 0.38029038558999695
Epoch : 8 Loss : 0.37456061213413877
Epoch : 9 Loss : 0.37898856362998484
Epoch : 10 Loss : 0.3687405374368032


In [84]:
# set model to eval mode
# we need to define our model explicitly you use as evalulate beacuse in deep leaarning some time behave diffrent during training and testing
# like dropout we fropout apply on only during traing we not apply dropout on testing
# same in case of the batch normalization


model.eval()

NN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [85]:
# Evaluation code: Evaluates the model on the test set

# Initialize variables to track the total number of samples and number of correct predictions
total = 0
correct = 0

# Disable gradient calculation to save memory and computation during evaluation
# `torch.no_grad()` ensures that gradients are not calculated during this phase
with torch.no_grad():

  # Iterate through each batch in the test_loader (this handles batching of test data)
  for batch_feature, label in test_loader:

    # Forward pass: Pass the batch of test features through the model to get the predictions (output)
    output = model(batch_feature)

    # Get the predicted class by selecting the index with the highest output score for each sample
    # `torch.max(output, 1)` returns the maximum value and its index along dimension 1 (the classes)
    _, predicted = torch.max(output, 1)

    # Update the total number of samples
    total = total + label.size(0)

    # Update the number of correct predictions by comparing predicted classes with actual labels
    correct = correct + (predicted == label).sum().item()

# Calculate and print the accuracy as the ratio of correct predictions to the total number of samples
print(f'Accuracy : {correct/total}')


Accuracy : 0.8668
